# KNN
Using two possible distance measures and rangeof k from 1 to 20 ( maybe can check overfitting with decreasing K)
try without standardization and try with mi-max (maybe try with Emmanuel's standardization)

In [1]:
import numpy as np
from utils_data_loading import*
from one_hot_encoding import*
from utils_features_manipulation import*
from utils_nans_manipulation import*

trainpath = 'Data/train.csv'
testpath = 'Data/test.csv'

## Importing train data and organizing

In [2]:
traindata,header = load_data(trainpath)

X_total, Y_total = structure_data(traindata)

## OHE
Before comparing distances it doesn't make sense to keep categorical variabes as integers instead OHE them. From EDA 3 variabes could be split and considered as categorical. PRI_jet_num definitely since there are only 4 unique values. And the centrality look like they could be split into two distributions with the threshold below.

DER_lep_eta_cenrality: nans, 0, 1 threshold 0.5 for 0 vs 1
DER_met_phi_centrality: threshold above or below 0


In [3]:
X_nans = replace_bad_data_with_nans(X_total, -999)

In [4]:
X_cleaned, nans_indices = replace_nans_with_median(X_nans, 0.5)

In [69]:
# Creating dict for indices
original_indices = {
    'jet' : header.index('PRI_jet_num')-1,
    'phi' : header.index('DER_met_phi_centrality')-1,
    'eta' : header.index('DER_lep_eta_centrality')-1
}

cat_indices = {}

for name,index in original_indices.items():
    if index not in nans_indices:
        skip = len([i for i in nans_indices if i < index])
        cat_indices[name] = index - skip

In [70]:
# Computing OHE for jets
jets = X_cleaned[:, cat_indices['jet']].astype(np.int8)
jets_OHE = get_jet_OHE(jets)

In [71]:
# Computing OHE for phi_centrality
phi_centrality = X_cleaned[:, cat_indices['phi']]
phi_centrality_OHE = get_centrality_split_OHE(phi_centrality, 0)

In [72]:
# Creating new matrix
X_OHE = np.c_[np.delete(X_cleaned, list(cat_indices.values()), axis=1), jets_OHE, phi_centrality_OHE]

##  Scaling non-categorical feature 
so that distances are comparable

In [74]:
# Init vars
tot_features = 31
cat_features = 2

X_scaled = X_OHE.copy()

last_non_cat = tot_features - cat_features # All OHE features are at the end

In [75]:
# Scale non categorical features
scalable_features = X_scaled[:,1:last_non_cat]  # Only scale non categorical feature

X_scaled[:,1:last_non_cat] = min_max_scale(scalable_features) 

In [111]:
# Move to a .py file I guess?

#pull maybe 1000 random samples
# Euclidean distance between two vectors
def dist_euclidean(row1, row2):
    r1=row1[1:] #takes features after column of 1
    r2=row2[1:]
    dist=np.linalg.norm(r1-rb)
    return dist

# calculate the manhattan distance between two vectors
def dist_manhattan(row1, row2):
    r1=row1[1:] #takes features after column of 1
    r2=row2[1:]
    dist=np.sum(np.abs(r1-r2))
    return dist

def dist_manhattan_broadcast(matrix, row):
    matrix = matrix[:,1:]
    row = row[1:]
    
    dist=np.sum(np.abs(matrix-row), axis=1)
    return dist

## KNN model

Looked at everything before this, cleaned the code a bit and made it more elegant, but it was actually already pretty good/fast (just some copy-paste errors). 

TODO: check KNN model func?

In [86]:
# TODO: delete

def split_data_tmp(x,y, ratio, seed=1):
    np.random.seed(seed)

    # split the data based on the given ratio: TODO    
    p = np.random.permutation(len(y))
    y = y[p]
    x = x[p]
    
    limit = int(len(y)*ratio)
        
    return x[:limit],y[:limit],x[limit:],y[limit:]

In [120]:
from time import time

def KNNmodel(Xtest,Xtrain,Ytrain,dist_func, k_n):
    """ input are extended X matrices, returns ypredictions array for Xtest using k closest distances
    calculated with given dist_func Given X's have been OHE and scales and y predictions are given as 0/1"""
    ypredarray=np.zeros(Xtest.shape[0])
    
    ret = np.empty((X_test.shape[0],k_n))
    np.r_[ret, indexn]
    
    for j in range(Xtest.shape[0]): #prediction for j'th row of Xtest
        rowdistances=np.zeros(Xtrain.shape[0])
        #for i in range(Xtrain.shape[0]):   #i denotes index of row of Xtrain
        
        rowdistances=dist_manhattan_broadcast(Xtrain, Xtest[j])  #computing distance between test point and each Xtrain row
        
        sortindex=np.argsort(rowdistances) #orders neighbor rows for closest distances 
        indexn=sortindex[:k_n]
        yneighbours=Ytrain[indexn]
        ypredclass=(yneighbours.mean()>0.5).astype(int)  #if average above 0.5 majority 1's classified as 1 otherwise classified as 0
        ypredarray[j]=ypredclass
    
    return ypredarray
        
    
    
    

In [ ]:
KNNmodel (...) -> (X_test, k)

KNN = KNNmodel(pessimistic_k)
for k in range(pessimistic_k)


In [ ]:
X_train, Y_train, X_test, Y_test = split_data_tmp(X_scaled, Y_total, 0.8)

KNNmodel(X_test, X_train, Y_train, dist_manhattan, 4)

NOTE
would it be possible to also (instead of splitting NAN - Non NANS and cleaning NAN's) Directly take X_total and clean NANs: remove features if more than 50% are NANs otherwise replace by median. 

^
To do this, just call: 'replace_nans_with_median(X_total, 0.5)'

# Old funcs

In [103]:
"""
#column indices. The headers as two extra coumn 'id' and 'predictions'
#X matrices have only 1 extra column of ones so indices should be 1 less than in the headers 
indice_OHE=[]
indice_OHE.append(header.index('DER_prodeta_jet_jet')-1)
indice_OHE.append(header.index('DER_prodeta_jet_jet')-1)   # Are these two names wrong?
indice_OHE.append(header.index('PRI_jet_num')-1)  # Shouldn't this be the first one?
cat=[2,2,4]   # What is this used for?


X_OHE=np.copy(X_total)  #X is the extanded "30+1 column array"

#JetNumOHE
colbefore=X_total[:,indice_OHE[0]] #extract column from Xtotal
classes=[0,1,2,3]
#initialize ohe sub-matrix for feature col
submatrix=np.zeros((X_total.shape[0], 4))
for i in range(4):
    coltmp=(colbefore==classes[i]).astype(int)
    submatrix[:,i]=coltmp
    
X_OHE=np.append(X_OHE, submatrix, axis=1)
    #remove column from XOHE
    #append new columns to XOHE

#NOTE: ONLY for X_Cleaned when no NAN values for the following two features. 
#Are these two features retained with the threshold 
#LepEtaCentrality OHE
colbefore=X_total[:,indice_OHE[1]]
submatrix=np.zeros((X_total.shape[0], 2))
coltmp0=(colbefore < 0.5).astype(int)
coltmp1=(colbefore >= 0.5).astype(int)
submatrix[:,0]=coltmp0
submatrix[:,1]=coltmp1
X_OHE=np.append(X_OHE, submatrix, axis=1)

#LepEtaCentrality OHE
colbefore=X_total[:,indice_OHE[2]]
submatrix=np.zeros((X_total.shape[0], 2))
coltmp0=(colbefore < 0).astype(int)
coltmp1=(colbefore >= 0).astype(int)
submatrix[:,0]=coltmp0
submatrix[:,1]=coltmp1
X_OHE=np.append(X_OHE, submatrix, axis=1)
    

#removing old features
    
X_OHE=np.delete(X_OHE, indice_OHE, axis=1)
"""

In [161]:
"""
Xscale=np.copy(X_OHE)
Original_number=31  #number of features before OHE +1 column
number_features_OHE=3 #again if the centrality features are still part of the X data array
newindex=Original_number-number_features_OHE
submatrix=Xscale[:,1:newindex]  #don't scale the 1 column and the OHE variables which start at index 31-3 because we removed 3 categorical variables
Xmin=(submatrix.min(axis=0)) #minimum array for the different features
Xmax=(submatrix.max(axis=0)) #maximum array for the different features
subscale=(submatrix-Xmin)/(Xmin-Xmax)
Xscale[1:newindex]=subscale
"""

ValueError: could not broadcast input array from shape (250000,27) into shape (27,36)